# <font color=red>__Démarche d'exploration et de visualisation des données__ </font>
## Exploration des données

Dans cette démonstration, on va explorer et analyser un dataset contenant les iformations des politiciens depuis 1980.
On va utiliser la bibliothéque pandas pour manipuler un fichier __CSV__ contenant ces données. 

In [16]:
import pandas as pd
import numpy as np

Notre dataset est donné sous forme de fichier CSV. Pandas fournit un moyen facile de lire notre fichier avec `read_csv`. Le chemin du fichier à lire est relatif à notre fichier sur le disque. Le chemin peut également être une URL, supportant HTTP, FTP et S3 si vos données sont stockées dans un seau AWS S3!

In [24]:
affairs = pd.read_csv('affairs.csv',nrows=5)
affairs.tail()

,sex,age,ym,child,religious,education,occupation,rate,nbaffairs
0,male,37,10.00,no,3,18,7,4,0
1,female,27,4.00,no,4,14,6,4,0
2,female,32,15.00,yes,1,12,1,4,0
3,male,57,15.00,yes,5,18,6,5,0
4,male,22,0.75,no,2,17,6,3,0


In [23]:
# Des variables sont inexploitables
# Choisir les colonnes utiles
affairs=pd.read_csv('affairs.csv',
    usecols=[1,2,4,6,8],nrows=5)
affairs.tail()

,age,ym,religious,occupation,nbaffairs
0,37,10.00,3,7,0
1,27,4.00,4,6,0
2,32,15.00,1,1,0
3,57,15.00,5,6,0
4,22,0.75,2,6,0


il est également possible  de laisser "tomber" les variable inexploitables:

In [14]:
df = affairs.drop(["nbaffairs", "rate"], axis=1)

df.head()

,sex,age,ym,child,religious,education,occupation
0,male,37,10.00,no,3,18,7
1,female,27,4.00,no,4,14,6
2,female,32,15.00,yes,1,12,1
3,male,57,15.00,yes,5,18,6
4,male,22,0.75,no,2,17,6


La première chose que nous allons vérifier est la taille de notre dataset. Nous pouvons utiliser `info ()` pour obtenir le nombre d'entrées de chaque colonne.

In [5]:
affairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 9 columns):
sex           601 non-null object
age           601 non-null float64
ym            601 non-null float64
child         601 non-null object
religious     601 non-null int64
education     601 non-null int64
occupation    601 non-null int64
rate          601 non-null int64
nbaffairs     601 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 42.3+ KB


In [ ]:
#nous pouvons redéfinir les types des variable lors de la lecture du fichier
df=pd.read_csv('affairs.csv',skiprows=1,header=None,usecols=[1,2,4,5,6],
  names=["sex","age","child","religious","education"],dtype={"sex":object,"age":float,"child":float,"religious":int,"education":object})
#redéfinition des bons types
df["sex"]=pd.Categorical(df["sex"],ordered=False)
df["age"]=pd.Categorical(df["age"],ordered=False)
df.dtypes

sex          category
age          category
child         float64
religious       int64
education      object
dtype: object

Alors, à quoi ressemblent les données. utilisons `head ()`, qui imprimera les 5 premières lign


In [ ]:
affairs.head()

,sex,age,ym,child,religious,education,occupation,rate,nbaffairs
0,male,37.0,10.00,no,3,18,7,4,0
1,female,27.0,4.00,no,4,14,6,4,0
2,female,32.0,15.00,yes,1,12,1,4,0
3,male,57.0,15.00,yes,5,18,6,5,0
4,male,22.0,0.75,no,2,17,6,3,0


## Echantillonnage simple
les fichiers de données sontsouvent trop gros, il reste possible, avant de chercher une configuration matérielle lourde et en première approximation, de tirer un échantillon aléatoire simple selon une distribution uniforme. Un tirage stratifié demanderait plus de travail. Cela suppose de connaître le nombre de ligne du fichier ou une valeur inférieure proche.

In [21]:
N=601  # taille du fichier
n=200  # taille de l'échantillon
lin2skipe=[0] # ne pas lire la première ligne
# ne pas lire N-n lignes tirées aléatoirement
lin2skipe.extend(np.random.choice(np.arange(1,N+1),
    (N-n),replace=False))
df_small=pd.read_csv("affairs.csv",
    skiprows=lin2skipe,header=None, 
    usecols=[1,2,3,4,5,6],
    names=["sex","age","ym","child",
         "religious","education"])
df_small

,sex,age,ym,child,religious,education
0,22.0,0.75,no,2,12,1
1,32.0,15.00,yes,4,16,1
2,27.0,4.00,no,4,14,5
3,37.0,15.00,yes,1,17,5
4,22.0,0.75,no,3,16,5


## Gérer un table de données
 ### Discrétisation d'une variable quantitative

In [ ]:
df["SexQ"]=pd.qcut(df.sex,2,labels=["male","female"])

#Modifier / regrouper des modalités
df["SexQ"]=df["SexQ"].cat.rename_categories(
    ["M","F"])
df["SexQ"].head()

0    F
1    M
2    M
3    F
4    M
Name: SexQ, dtype: category
Categories (2, object): [M < F]

In [7]:
affairs['sex'].head()

0      male
1    female
2    female
3      male
4      male
Name: sex, dtype: object

In [8]:
affairs['sex'].value_counts()

female    315
male      286
Name: sex, dtype: int64

Nous pouvons utiliser 'describe()' pour visualiser quelques données statistiques

In [9]:
affairs['age'].describe()

count    601.000000
mean      32.487521
std        9.288762
min       17.500000
25%       27.000000
50%       32.000000
75%       37.000000
max       57.000000
Name: age, dtype: float64

In [10]:
affairs['age'].max()

57.0

In [11]:
affairs.describe()

,age,ym,religious,education,occupation,rate,nbaffairs
count,601.000000,601.000000,601.000000,601.000000,601.000000,601.000000,601.000000
mean,32.487521,8.177696,3.116473,16.166389,4.194676,3.931780,1.455907
std,9.288762,5.571303,1.167509,2.402555,1.819443,1.103179,3.298758
min,17.500000,0.125000,1.000000,9.000000,1.000000,1.000000,0.000000
25%,27.000000,4.000000,2.000000,14.000000,3.000000,3.000000,0.000000
50%,32.000000,7.000000,3.000000,16.000000,5.000000,4.000000,0.000000
75%,37.000000,15.000000,4.000000,18.000000,6.000000,5.000000,0.000000
max,57.000000,15.000000,5.000000,20.000000,7.000000,5.000000,12.000000


In [12]:
affairs[affairs['sex'] == 'female'].head()

,sex,age,ym,child,religious,education,occupation,rate,nbaffairs
1,female,27.0,4.00,no,4,14,6,4,0
2,female,32.0,15.00,yes,1,12,1,4,0
5,female,32.0,1.50,no,2,17,5,5,0
6,female,22.0,0.75,no,2,12,1,3,0
8,female,32.0,15.00,yes,4,16,1,2,0


L'instruction ci-dessus renvoie une nouvelle image (pas une copie, la modification de cette donnée modifiera aussi l'original).

In [13]:
affairs[affairs['sex'] == 'female'].describe()

,age,ym,religious,education,occupation,rate,nbaffairs
count,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000
mean,30.801587,8.017070,3.107937,15.257143,3.384127,3.939683,1.419048
std,8.646241,5.627237,1.134758,2.023879,1.929094,1.148045,3.309264
min,17.500000,0.125000,1.000000,9.000000,1.000000,1.000000,0.000000
25%,22.000000,1.500000,2.000000,14.000000,1.000000,3.000000,0.000000
50%,27.000000,7.000000,3.000000,16.000000,4.000000,4.000000,0.000000
75%,37.000000,15.000000,4.000000,17.000000,5.000000,5.000000,0.000000
max,57.000000,15.000000,5.000000,20.000000,7.000000,5.000000,12.000000


Nous pouvons également créer de nouvelles lignes. Nous allons créer une nouvelle colonne contenant True ou False, que la personne soit inférieure à 30 ou non.

In [ ]:
affairs['below_30'] = affairs['age'] < 30

In [15]:
affairs['below_30'].value_counts()

False    325
True     276
Name: below_30, dtype: int64

In [16]:
affairs.head()

,sex,age,ym,child,religious,education,occupation,rate,nbaffairs,below_30
0,male,37.0,10.00,no,3,18,7,4,0,False
1,female,27.0,4.00,no,4,14,6,4,0,True
2,female,32.0,15.00,yes,1,12,1,4,0,False
3,male,57.0,15.00,yes,5,18,6,5,0,False
4,male,22.0,0.75,no,2,17,6,3,0,True


We can use this to normalize our columns with better values. Take for example `religious`. The number have the following meaning: 1 = not, 2 = mildly, 3 = fairly, 4 = strongly. We can easily replace them inline with the following code.

In [17]:
rel_meanings = ['not', 'mildly', 'fairly', 'strongly']

In [18]:
affairs['religious'] = affairs['religious'].apply(lambda x: rel_meanings[min(x, 4)-1])

In [19]:
affairs.head()

,sex,age,ym,child,religious,education,occupation,rate,nbaffairs,below_30
0,male,37.0,10.00,no,fairly,18,7,4,0,False
1,female,27.0,4.00,no,strongly,14,6,4,0,True
2,female,32.0,15.00,yes,not,12,1,4,0,False
3,male,57.0,15.00,yes,strongly,18,6,5,0,False
4,male,22.0,0.75,no,mildly,17,6,3,0,True


## 2-Fichier JSON

Dans cette section on va explorer de données d'un fichier JSON, les fichiers de données de grandes tailles peuvent être pénibles pour les charger en mémoire. Dans ce cas, une combinaison d'outils de ligne de commande et de Python peut constituer un moyen efficace d'__explorer__ et d'analyser les données.

On va travailler sur un fichier JSON contenant les données des activités des agents de police concernant les violations de la circulation en Montgomery, Maryland.Les données contiennent des informations sur l'endroit où la violation s'est produite, le type de voiture, les données démographiques sur la personne qui reçoit la violation, et d'autres informations intéressantes.

In [2]:
%%bash

head md_traffic.json

{
  "meta" : {
    "view" : {
      "id" : "4mse-ku6q",
      "name" : "Traffic Violations",
      "averageRating" : 0,
      "category" : "Public Safety",
      "createdAt" : 1403103517,
      "description" : "This dataset contains traffic violation information from all electronic traffic violations issued in the County.  Any information that can be used to uniquely identify the vehicle, the vehicle owner or the officer issuing the violation will not be published.\r\n\r\nUpdate Frequency:  Daily",
      "displayType" : "table",
